In [1]:
import json
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from pprint import pprint

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

from datasets import load_dataset, load_from_disk
from transformers import (
    AutoTokenizer,
    BertModel, BertPreTrainedModel,
    AdamW, get_linear_schedule_with_warmup,
    TrainingArguments, AutoModel,
)

In [2]:
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

PyTorch version:[1.7.1].
device:[cuda:0].


In [8]:
train_dataset = load_from_disk("../data/train_dataset/train")
sample_idx = np.random.choice(range(len(train_dataset)), 30)
training_dataset = train_dataset[sample_idx]
print(training_dataset['context'][0])
print(training_dataset['question'][0])
# sample_idx

이 문서는 브르타뉴 공국의 통치자 목록이다. 브르타뉴 군주들은 왕, 제후, 공작 등 시대 별로 달랐다. 브르타뉴 통치자들은 어쩔 때는 선출되기도 하고, 어쩔 때는 정복이나 계략, 상속권을 통해 자리에 올랐다. 세습 공작 중에는 때로는 여성 통치자가 있기도 했으며, 브르타뉴 여공작이라는 작위를 지녔다. 주요 도시들과 지역들은 브르타뉴 통치자들과 자주 분쟁을 벌이거나 브르타뉴 통치자가 된 백작들의 지배를 받았다.\n\n로마 제국이 쇠퇴해가던 시절에, 갈리아내 초기 브르타뉴인은 코르누아이와 돔노니아 등의 작은 왕국들의 왕이라고 자청했다. 이런 왕들 중에 일부는 아르모리카 반도내에 브리튼족들에 대한 헤게모니를 형성했을 것이고, 요르다네스에 대해 연대기 작가 요르다네스는 브리튼족의 왕이라 칭해다. 그럼에도 브르타뉴 전체에 대한 통치자는 없었으며, 브르타뉴는 지역 백작들의 영지들로 나누어졌다.\n\n브르타뉴 공국은 939년 트랑라포레 전투에서 기원을 했으며, 브르타뉴와 노르망디 간에 경계인 쿠에농강에 세워졌다. 942년, 알랑 2세는 루이 4세에게 충성을 서약했으나, 브르타뉴 공국은 루이 6세가 낭트 주교가 된 1123년까지 프랑스 왕가의 관심을 갖기 못했다. 다른 어떤 브르타뉴 공작들도 아르튀르 1세가 1202년에 필리프 2세를 주군이라 인정할 때까지 알랑 2세의 충성 맹세를 되풀이하지 않았다. \n\n브르타뉴 지역은 흔히 공국이라 불렸고, 브르타뉴의 통치자도 독립 주권을 지닌 공작으로서 여겨졌다. 하지만 한 역사적 관점은 12세기 중반 이전에 당시 프랑스 왕국은 브르타뉴를 더 이상 영지라고 보지 않았기에 브르타뉴 공작들을 프랑스 왕들이 백작이라 불렀다고도 본다.날짜=2018-10-30 1297년에 브르타뉴는 프랑스의 대귀족제에서 공국으로 승격되었다. 이런 태도는 샤를 8세, 그 다음에는 루이 12세가 브르타뉴와 상속을 위해 각각 결혼한 안 드 브르타뉴의 상속권에 접근했던 방식과는 모순된다.
브르타뉴와 노르망디를 구분짓는 것은?


In [4]:
class RobertaEncoder(AutoModel):
    def __init__(self, config):
        super(RobertaEncoder, self).__init__(config)

        self.roberta = AutoModel(config)
        self.init_weights()
      
    def forward(self, input_ids,  attention_mask=None): 

        outputs = self.roberta(input_ids, attention_mask=attention_mask)
        
        pooled_output = outputs[1]
        return pooled_output

In [5]:
model_checkpoint = "xlm-roberta-base"
p_encoder = RobertaEncoder.from_pretrained(model_checkpoint)
q_encoder = RobertaEncoder.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
    p_encoder.cuda()
    q_encoder.cuda()

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [7]:
class DenseRetrieval:
    def __init__(self, args, dataset, num_neg, tokenizer, p_encoder, q_encoder):

        self.args = args
        self.dataset = dataset
        self.num_neg = num_neg

        self.tokenizer = tokenizer
        self.p_encoder = p_encoder
        self.q_encoder = q_encoder

        self.prepare_in_batch_negative(num_neg=num_neg)
        
        # self.eval_dataset = eval_dataset
        # self.prepare_eval_dataset

    def prepare_in_batch_negative(self, dataset=None, num_neg=2, tokenizer=None):

        if dataset is None:
            dataset = self.dataset

        if tokenizer is None:
            tokenizer = self.tokenizer

        # 1. In-Batch-Negative 만들기
        # CORPUS를 np.array로 변환해줍니다.
        corpus = np.array(list(set([example for example in dataset["context"]])))
        p_with_neg = []

        for c in dataset["context"]:
            while True:
                neg_idxs = np.random.randint(len(corpus), size=num_neg)

                if not c in corpus[neg_idxs]:
                    p_neg = corpus[neg_idxs]

                    p_with_neg.append(c)
                    p_with_neg.extend(p_neg)
                    break

        # 2. (Question, Passage) 데이터셋 만들어주기
        q_seqs = tokenizer(dataset["question"], padding="max_length", truncation=True, return_tensors="pt")
        p_seqs = tokenizer(p_with_neg, padding="max_length", truncation=True, return_tensors="pt")

        max_len = p_seqs["input_ids"].size(-1)
        p_seqs["input_ids"] = p_seqs["input_ids"].view(-1, num_neg+1, max_len)
        p_seqs["attention_mask"] = p_seqs["attention_mask"].view(-1, num_neg+1, max_len)
        # p_seqs["token_type_ids"] = p_seqs["token_type_ids"].view(-1, num_neg+1, max_len)

        train_dataset = TensorDataset(
            p_seqs["input_ids"], p_seqs["attention_mask"], # p_seqs["token_type_ids"], 
            q_seqs["input_ids"], q_seqs["attention_mask"], # q_seqs["token_type_ids"]
        )

        self.train_dataloader = DataLoader(
            train_dataset,
            shuffle=True,
            batch_size=self.args.per_device_train_batch_size
        )

        valid_seqs = tokenizer(
            dataset["context"],
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        passage_dataset = TensorDataset(
            valid_seqs["input_ids"],
            valid_seqs["attention_mask"],
            # valid_seqs["token_type_ids"]
        )
        self.passage_dataloader = DataLoader(
            passage_dataset,
            batch_size=self.args.per_device_train_batch_size
        )


    def train(self, args=None):
        if args is None:
            args = self.args
        batch_size = args.per_device_train_batch_size
        print(args.device)

        # Optimizer
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {"params": [p for n, p in self.p_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
            {"params": [p for n, p in self.p_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
            {"params": [p for n, p in self.q_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
            {"params": [p for n, p in self.q_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0}
        ]
        optimizer = AdamW(
            optimizer_grouped_parameters,
            lr=args.learning_rate,
            eps=args.adam_epsilon
        )
        t_total = len(self.train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=args.warmup_steps,
            num_training_steps=t_total
        )

        # Start training!
        global_step = 0

        self.p_encoder.zero_grad()
        self.q_encoder.zero_grad()
        torch.cuda.empty_cache()

        train_iterator = tqdm(range(int(args.num_train_epochs)), desc="Epoch")
        # for _ in range(int(args.num_train_epochs)):
        for _ in train_iterator:

            with tqdm(self.train_dataloader, unit="batch") as tepoch:
                for batch in tepoch:

                    p_encoder.train()
                    q_encoder.train()
            
                    targets = torch.zeros(batch_size).long() # positive example은 전부 첫 번째에 위치하므로
                    targets = targets.to(args.device)

                    p_inputs = {
                        "input_ids": batch[0].view(batch_size * (self.num_neg + 1), -1).to(args.device),
                        "attention_mask": batch[1].view(batch_size * (self.num_neg + 1), -1).to(args.device),
                        # "token_type_ids": batch[2].view(batch_size * (self.num_neg + 1), -1).to(args.device)
                    }
            
                    q_inputs = {
                        "input_ids": batch[2].to(args.device),
                        "attention_mask": batch[3].to(args.device),
                        # "token_type_ids": batch[5].to(args.device)
                    }

                    # (batch_size*(num_neg+1), emb_dim)
                    p_outputs = self.p_encoder(**p_inputs)[1]
                    # (batch_size, emb_dim)
                    q_outputs = self.q_encoder(**q_inputs)[1]

                    # Calculate similarity score & loss
                    # p_outputs = p_outputs.view(batch_size, -1, self.num_neg+1)
                    p_outputs = torch.transpose(p_outputs.view(batch_size, self.num_neg+1, -1), 1, 2)
                    q_outputs = q_outputs.view(batch_size, 1, -1)

                    sim_scores = torch.bmm(q_outputs, p_outputs).squeeze()  #(batch_size, num_neg + 1)
                    sim_scores = sim_scores.view(batch_size, -1)
                    sim_scores = F.log_softmax(sim_scores, dim=1)

                    loss = F.nll_loss(sim_scores, targets)
                    tepoch.set_postfix(loss=f"{str(loss.item())}")

                    loss.backward()
                    optimizer.step()
                    scheduler.step()

                    self.p_encoder.zero_grad()
                    self.q_encoder.zero_grad()

                    global_step += 1

                    torch.cuda.empty_cache()

                    del p_inputs, q_inputs


    def get_relevant_doc(self, query, k=1, args=None, p_encoder=None, q_encoder=None):
    
        if args is None:
            args = self.args

        if p_encoder is None:
            p_encoder = self.p_encoder

        if q_encoder is None:
            q_encoder = self.q_encoder

        with torch.no_grad():
            p_encoder.eval()
            q_encoder.eval()

            q_seqs_val = self.tokenizer(
                [query],
                padding="max_length",
                truncation=True,
                return_tensors="pt"
            ).to(args.device)
            q_emb = q_encoder(**q_seqs_val)[1].to("cpu")  # (num_query=1, emb_dim)

            p_embs = []
            for batch in tqdm(self.passage_dataloader):

                batch = tuple(t.to(args.device) for t in batch)
                p_inputs = {
                    "input_ids": batch[0],
                    "attention_mask": batch[1],
                    # "token_type_ids": batch[2]
                }
                p_emb = p_encoder(**p_inputs)[1].to("cpu")
                p_embs.append(p_emb)

        # (num_passage, emb_dim)
        p_embs = torch.stack(p_embs, dim=0).view(len(self.passage_dataloader.dataset), -1)

        dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
        rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
        # print(dot_prod_scores)
        # print(rank)

        return rank[:k]


In [9]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    weight_decay=0.01,
)

In [10]:
retrieval = DenseRetrieval(args=args, 
                           dataset=training_dataset,
                           # eval_dataset=None, 
                           num_neg=2, 
                           tokenizer=tokenizer, 
                           p_encoder=p_encoder, 
                           q_encoder=q_encoder
                          )

In [11]:
retrieval.train()

cuda:0


In [14]:
IDX = 1
print(training_dataset['question'][IDX])
print(training_dataset['context'][IDX])

러일 전쟁의 승전국은?
전라남도 광주 출신이다. 출생 연도나 성장 과정에 대해서는 알려지지 않았다.\n\n러일 전쟁이 일본 제국의 승리로 끝나고 을사조약이 체결되자, 전국적으로 을사조약에 반대하는 을사의병이 일어났다. 임창모는 전남 화순군 능주 출신인 양회일과 함께 의병을 일으켰다. 1907년에 화순에서 공격을 당하여 포위되었고, 의병을 이끌던 양회일과 임창모가 모두 체포되었다. \n\n체포된 두 사람은 광주감옥에 갇혔다. 양회일은 단식투쟁을 하다가 곧 옥사하였고, 임창모는 신안군 지도읍 지도로 유배되었다. 이듬해인 1908년에 유배가 해제되어 풀렸났다. \n\n임창모는 유배에서 풀려나자마자 당시 세력을 떨치고 있던 전남 보성군의 안규홍 의병대에 합류하여 다시 의병 운동에 뛰어들었다. 1908년 6월 이후에는 임창모의 부대가 안규홍의 의병대에서 따로 분리되어 독립하였으며, 보성에 거점을 두고 활동하였다.\n\n1909년 10월에 호남 지역에서 군과 경찰이 투입되는 대대적인 토벌 작전이 펼쳐졌다. 임창모는 전남 해남군과 강진군, 영암군에 걸쳐 있는 흑석산에서 첫째아들 임학규와 함께 있다가 정체가 발각되었고, 병부대는 기습을 받아 전멸하고 임창모도 전사하였다.\n\n1963년에 건국훈장 독립장이 추서되었다.


In [15]:
print(training_dataset['question'][IDX])
# print(retrieval.get_relevant_doc(training_dataset['question'][IDX]))
retrieval_list = retrieval.get_relevant_doc(training_dataset['question'][IDX], k=5)
for i in retrieval_list.tolist():
    print(training_dataset['context'][i])

러일 전쟁의 승전국은?



이 밴드의 이전 음반인 1989년 《Mother's Milk》는 빌보드 200에 진입한 두 번째 음반이 되었으며, 이 음반은 52위에 그 당시 그들의 생애에서 가장 큰 음반이었다. 비록 음반은 다소 성공적이었지만, 제작은 프로듀서 마이클 베인혼에 의해 짓눌렸다. 그는 프루시안테에게 전체적으로 더 무거운 음색으로 연주하도록 설득했고, 앤서니 키디스에게 더 라디오가 가능한 가사를 쓰라고 지시하여 밴드가 창작적으로 제약을 느끼게 했다. \n\nEMI와 밴드의 계약이 끝나면서 그들은 또 다른 음반사를 찾기 시작했다. 이 그룹은 소니 BMG/에픽과 함께 EMI로부터 마지막 음반을 구입한다는 단서로 함께 가기로 합의했다. 레이블은 며칠밖에 걸리지 않을 것이라고 약속했지만, 그 과정은 수개월로 연장되었다 비록 소니/에픽과 거래가 이루어졌지만, 워너 브라더스 레코드의 모 오스틴은 키디스에게 전화를 걸어 성공적인 계약을 축하했고, 라이벌 음반사를 칭찬했다. 키디스는 당시 상황을 회상하며 "우리가 이 모든 협상 중에 만났던 가장 멋지고, 가장 실제적인 인물은 제가 경쟁 회사에 훌륭한 기록을 세우도록 격려하기 위해 방금 개인적으로 전화를 했습니다. 그런 사람이면 내가 일하고 싶은 사람이었어요." 이 그룹은 이 아이디어를 추구했고, 결국 워너 브라더스와의 계약에 찬성하여 소니와의 계약을 포기했다. 오스틴은 EMI의 옛 친구에게 전화를 걸었는데, EMI는 즉시 레이블 이전을 허락했다
통킹 사건\n3월 28일, 브리에르 드 리즐이 파리에 보낸 비관적인 ‘랑선 전보’는 막대한 정치적 파장을 낳았다. 페리의 즉각적인 반응은 통킹에서 군대를 증원한다는 것이었다. 실제로 브리에르 드 리즐은 상황에 대한 그의 추정치를 빠르게 수정하고 정부에게 전선을 곧 안정화시킬 수 있다고 조언했다. 그러나 그의 두 번째 생각은 너무 늦었다. 그의 첫 전보가 파리에 공개되자, 하원에서는 소란이 일었다. 조르주 클레망소가 이끄는 정부의 반대파들이 공격을 시작했다. 불신임 동의안이 상정되었고, 페리 정부는 3월

In [ ]:
# 이거 에폭마다 eval 진행하게 함수 추가하기